**Как хранятся графы?**

В виде матрицы смежности A хранить граф глупо, так как данные почти всегда разреженные. Поэтому всегда хранится в виде sparse массива - только ненулевые элементы. Это то же самое, что хранить просто в виде таблицы:

|Source Vertex|Destination Vertex|
|---|---|
|1|2|
|1|3|
|2|4|
|3|5|
|4|6|

Деревья (специальный вид графов) так же хранятся.

У реальных графов всегда есть атрибуты вершины и атрибуты ребер

|Source Vertex|Dest Vertex|Edge Properties|
|---|---|---|
|1|2|(relation='is a friend of', recency=5)|
|1|3|(relation='follows', recency=10)|
|3|5|(relation='foolows', recency=2)|


Кроме того нужна структура для хранения атрибутов вершин. Для этого ведется отдельная таблица:

|Vertex|Vertex Properties|
|---|---|
|1|(name='Alex', surname='Pompeo')|
|2|(name='John', surname='Bombaleilo')|
|3|(name='Katrin', surname='Zeta')|

## Spark GraphX

#### Как подключить
import org.apache.spark.graphx._

Специальные структуры для работы с графами не требуются, достаточно обычного RDD.

#### Примеры задач, которые можно решать анализируя корпус текстов Wikipedia:
- *PageRank* - наиболее значимые страницы
- *Topic Definition* - выделение тематик
- *User Communities* - выявление сообществ
- *Community Topics* - определение тематик сообществ
<img src="img/graphx_tasks.png" width=500>

#### Иерархия базовых классов в GraphX

val graph = Graph(VertexRDD, EdgeRDD)

*EdgeRDD* = RDD[Edge] с возможной перегруппировкой по партициям и 3 встроенными функциями

*VertexRDD* = RDD[(VertexID,VD)] с ограничением на уникальность VertexID и несколькими встроенными функциями

|method|desc|
|:--|:--|
|edges()|RDD с ребрами|
|vertices()|RDD с вершинами|
|numEdges()|кол-во ребер|
|numVertices()|кол-во вершин|
|||
|mapVertices(f)|обработать вершины|
|mapEdges(f)| обработать ребра|
|mapTriplets(f)| обработать обогащенные ребра|
|||
|aggregateMessages| считает показатель для каждой связи и агрегирует его по dest ребра |
|mapReducetriplets| то же самое, но в другой нотации |
|collectNeighborIds| возвращает RDD со списком соседей для каждой вершины|
|collectNeighbors| возвращает RDD со списком и атирбутами соседей для каждой вершинам|
|||

Класс EdgeTriplet - просто ребро со всеми атрибутами (соединенный Edge с Vertex)

<img src="img/graphx_classes.png">

aggregateMessages(send_f,merge_f) - .


#### Storage

Как бить граф по нодам кластера?

Можно выбрать стратегию партичионирования с помощью partitionBy().
- 2D parition

Ниже приведен пример для 2х партиций. Каждое ребро в своей партиции. Некоторые вершины дублируются.
<img src="img/graphx_partition.png" width=450>

#### Какие есть готовые алгоритмы
- In-Degrees
- PageRank
- Triangles
- Connected Components

#### Как генерируются графы в GraphX?
- FromEdgeList
- Lognormal

#### Резюме
Максмимально базовая библиотека, из готового нет почти ничего. Единственное преимущество - возможность распределения вычислений.




# Pregel
В 2010 году Google разработал Pregel - парадигму вычислений на графах. В некотором смысле она является альтернативой MapReduce. Позднее концепция перекочевала в Apache Giraf (который использовал Facebook) и собственно Apache Spark GraphX.

**Идея:** вычисления выполняются в параллель для каждой вершины графа. Каждая такая итерация называется superstep. 

Выход итерации: обновленный граф (state) + **сообщения** с неким промежуточным результатом (messages), которые отсылаются в соседние вершины и затем там агрегируются.

Схема представлена ниже:

<img src="img/graphx_pregel.png" width=500>

#### Примеры
1. Посчитать максимум по всем нодам
    - отправляем свое текщее значение соседям
    - если в полученных от соседей сосбщениях есть значение больше, то обновить свое и отправить его соседям еще раз
    - если все значения меньше, делаем halt
    <img src="img/graphx_max.png" width=250>
2. Посчитать средний возраст подписчиков (из примеров SparkX)
    - функция map отправляет сообщение c 1 по направлению текущей связи
    - функция reduce суммирует по каждой ноде все сообщения пришедшие в нее
    - за одну итерацию посчитали
3. PageRank
    - то же самое, но в цикле


плюс любые алгоритмы, связанные с переходом по графу (практически все).


Основное отличие от MapReduce - там между партициями должен синхронизироваться результат (данные), а в Pregel пересылаются только сообщения, сами данные сохраняются по партициям.

В GraphX pregel-сообщение - это просто итератор с двумя параметрами (dstId, attrs).